In [2]:
## This file uses all generated data, so it needs to be run after the files that use kaggle data

In [3]:
import numpy as np
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [81]:
coach_features = pd.read_csv('data/generated_data/coach_features.csv')
stationary_probs_all = pd.read_csv('data/generated_data/stationary_probabilities.csv')
stationary_probs = stationary_probs_all[stationary_probs_all['Season'] >= 2003]
team_season_features = pd.read_csv('data/generated_data/team_season_features.csv')
additional_team_features = pd.read_csv('data/generated_data/additional_team_features.csv')
preseason_rankings = pd.read_csv('data/generated_data/preseason_rankings.csv')

In [82]:
print(coach_features.shape)
coach_features.head()

(6959, 11)


,TeamID,Season,school,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car
0,1102,2003,air-force,False,False,False,0.341,0,0,0,0
1,1103,2003,akron,False,False,False,0.462,0,0,0,0
2,1104,2003,alabama,False,True,False,0.660,4,0,0,0
3,1105,2003,alabama-am,False,False,False,0.518,0,0,0,0
4,1412,2003,alabama-birmingham,False,False,False,0.618,0,0,0,0


In [83]:
print(stationary_probs.shape)
stationary_probs.head()

(6892, 4)


,Unnamed: 0,Season,TeamID,pi_i
5407,5407,2003,1102,0.002807
5408,5408,2003,1103,0.002684
5409,5409,2003,1104,0.004745
5410,5410,2003,1105,0.001239
5411,5411,2003,1106,0.001900


In [84]:
print(team_season_features.shape)
team_season_features.head()

(6884, 38)


,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement
0,2003,1102,28,12,16,0.428571,57.250000,57.000000,0.250000,15.878163,3.0,4.0,0.0,2.0,20.821429,7.821429,0.375643,11.142857,17.107143,0.651357,11.428571,52.232914,109.273561,109.522922,0.515214,-0.086643,None,1.0,0.0,7.0,9.0,8.0,158.0,149.0,-9.0,160.0,151.0,-9.0
1,2003,1103,27,13,14,0.481481,78.777778,78.148148,0.629630,11.126039,4.0,5.0,4.0,6.0,16.074074,5.444444,0.338710,19.037037,25.851852,0.736390,12.629630,64.917792,116.651118,116.489034,0.527877,-0.046395,None,1.0,1.0,1.0,13.0,2.0,182.0,198.0,16.0,163.0,187.0,24.0
2,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,6.357143,0.320144,14.857143,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,Y10,5.0,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0
3,2003,1105,26,7,19,0.269231,71.769231,76.653846,-4.884615,15.552212,2.0,7.0,1.0,7.0,20.769231,7.576923,0.364815,15.423077,21.846154,0.705986,18.653846,71.461497,97.856445,106.384802,0.285796,-0.016565,None,1.0,0.0,3.0,16.0,4.0,313.0,316.0,3.0,307.0,313.0,6.0
4,2003,1106,28,13,15,0.464286,63.607143,63.750000,-0.142857,12.374266,5.0,4.0,2.0,5.0,17.642857,6.107143,0.346154,10.642857,16.464286,0.646421,17.035714,64.071390,98.405306,99.544614,0.492199,-0.027913,None,1.0,0.0,4.0,11.0,5.0,248.0,236.0,-12.0,263.0,227.0,-36.0


In [85]:
print(additional_team_features.shape)
additional_team_features.head()

(1274, 7)


,TeamID,Season,school,returning_min_pct,returning_score_pct,avg_height,avg_yr
0,1103,2022,akron,60.1,53.1,76,2.2
1,1104,2022,alabama,46.6,49.3,77,1.3
2,1412,2022,alabama-birmingham,60.0,63.2,77,3.1
3,1112,2022,arizona,48.3,45.9,80,1.5
4,1116,2022,arkansas,37.5,37.4,77,2.5


In [86]:
idx = ['Season', 'TeamID']
features_all = team_season_features.merge(stationary_probs, on=idx)\
.merge(coach_features, on=idx)\
.merge(additional_team_features, on=idx)\
.merge(preseason_rankings, on=idx, how='left')
features_all['preseason_pts'] = features_all['preseason_pts'].fillna(0)
features_all.head()

,Season,TeamID,total_games,num_wins,num_losses,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3FGM_pg,3_pct,FTM_pg,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,Seed,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,prev_net,net_improvement,final_pom,prev_pom,pom_improvement,Unnamed: 0,pi_i,school_x,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,school_y,returning_min_pct,returning_score_pct,avg_height,avg_yr,school,preseason_pts
0,2003,1104,28,17,11,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,6.357143,0.320144,14.857143,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,Y10,5.0,1.0,6.0,5.0,11.0,38.0,27.0,-11.0,33.0,39.0,6.0,5409,0.004745,alabama,False,True,False,0.660,4,0,0,0,alabama,80.5,78.0,76,1.8,alabama,1295.0
1,2003,1112,28,25,3,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,7.035714,0.350534,17.535714,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,Z01,10.0,5.0,2.0,1.0,12.0,2.0,8.0,6.0,3.0,3.0,0.0,5416,0.008013,arizona,True,False,False,0.738,24,12,5,1,arizona,98.4,99.0,77,1.6,arizona,1773.0
2,2003,1113,29,18,11,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,4.000000,0.317808,17.551724,26.206897,0.669737,14.000000,65.671136,115.334912,103.952243,0.786348,-0.165659,Z10,4.0,2.0,10.0,1.0,14.0,32.0,30.0,-2.0,32.0,24.0,-8.0,5417,0.004979,arizona-state,False,False,False,0.520,3,0,0,0,arizona-state,73.2,66.6,77,1.6,arizona state,4.0
3,2003,1120,30,19,11,0.633333,70.100000,65.566667,4.533333,18.358891,6.0,4.0,2.0,2.0,16.333333,5.633333,0.344898,13.066667,19.733333,0.662162,15.900000,64.715520,108.023523,101.505949,0.717067,-0.083733,W10,4.0,1.0,8.0,3.0,12.0,36.0,31.0,-5.0,45.0,45.0,0.0,5423,0.004402,auburn,False,False,False,0.617,8,3,0,0,auburn,67.8,70.9,77,1.6,NaN,0.0
4,2003,1122,27,20,7,0.740741,69.703704,67.703704,2.000000,12.584529,9.0,0.0,8.0,1.0,17.185185,6.000000,0.349138,16.592593,24.000000,0.691358,15.222222,63.424000,108.269069,104.582414,0.599878,0.140863,W13,1.0,1.0,1.0,6.0,2.0,73.0,95.0,22.0,123.0,143.0,20.0,5424,0.003029,austin-peay,False,False,False,0.522,2,0,0,0,austin-peay,73.5,62.7,77,1.2,NaN,0.0


In [87]:
print(features_all.shape)

(1284, 56)


In [88]:
features_all[features_all['Season'] == 2022].sort_values(by='pi_i', ascending=False)[['school_y', 'pi_i']].head(50)

,school_y,pi_i
1233,gonzaga,0.008481
1241,kentucky,0.008245
1217,arizona,0.007702
1234,houston,0.007476
1240,kansas,0.007394
1219,auburn,0.007352
1220,baylor,0.007277
1259,purdue,0.007019
1278,villanova,0.006919
1272,texas-tech,0.006868


In [89]:
features_all.columns

Index(['Season', 'TeamID', 'total_games', 'num_wins', 'num_losses', 'win_pct',
       'avg_points_for', 'avg_points_against', 'avg_game_margin',
       'std_game_margin', 'non_home_wins', 'home_losses', 'close_wins',
       'close_losses', '3FGA_pg', '3FGM_pg', '3_pct', 'FTM_pg', 'FTA_pg',
       'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff', 'pythag_win_pct',
       'luck', 'Seed', 'wins_vs_tourney_teams', 'away_wins_vs_tourney_teams',
       'losses_vs_tourney_teams', 'losses_vs_non_tourney_teams',
       'games_vs_tourney_teams', 'final_net', 'prev_net', 'net_improvement',
       'final_pom', 'prev_pom', 'pom_improvement', 'Unnamed: 0', 'pi_i',
       'school_x', 'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'school_y', 'returning_min_pct', 'returning_score_pct', 'avg_height',
       'avg_yr', 'school', 'preseason_pts'],
      dtype='object')

In [90]:
features = features_all[['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for', 
              'avg_points_against', 'avg_game_margin', 'std_game_margin', 
              'non_home_wins', 'home_losses', 'close_wins', 'close_losses', 
              '3FGA_pg', '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 
              'off_eff', 'def_eff', 'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
              'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
              'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
              'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
              'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
              'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
              'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr', 'preseason_pts']]

In [92]:
print(features.shape)
features.head()

(1284, 46)


,Season,TeamID,Seed,total_games,win_pct,avg_points_for,avg_points_against,avg_game_margin,std_game_margin,non_home_wins,home_losses,close_wins,close_losses,3FGA_pg,3_pct,FTA_pg,FT_pct,TOV_pg,avg_pace,off_eff,def_eff,pythag_win_pct,luck,wins_vs_tourney_teams,away_wins_vs_tourney_teams,losses_vs_tourney_teams,losses_vs_non_tourney_teams,games_vs_tourney_teams,final_net,net_improvement,final_pom,pom_improvement,pi_i,is_ap_pre_top_5,is_ap_pre_top_15,is_ap_pre_top_25,coach_WL_car,tourneys_car,sw16_car,ff_car,champ_car,returning_min_pct,returning_score_pct,avg_height,avg_yr,preseason_pts
0,2003,1104,Y10,28,0.607143,69.285714,65.000000,4.285714,13.149843,4.0,2.0,1.0,2.0,19.857143,0.320144,20.928571,0.709898,13.285714,63.168152,108.882967,103.099630,0.708513,-0.101370,5.0,1.0,6.0,5.0,11.0,38.0,-11.0,33.0,6.0,0.004745,False,True,False,0.660,4,0,0,0,80.5,78.0,76,1.8,1295.0
1,2003,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.008013,True,False,False,0.738,24,12,5,1,98.4,99.0,77,1.6,1773.0
2,2003,1113,Z10,29,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,0.317808,26.206897,0.669737,14.000000,65.671136,115.334912,103.952243,0.786348,-0.165659,4.0,2.0,10.0,1.0,14.0,32.0,-2.0,32.0,-8.0,0.004979,False,False,False,0.520,3,0,0,0,73.2,66.6,77,1.6,4.0
3,2003,1120,W10,30,0.633333,70.100000,65.566667,4.533333,18.358891,6.0,4.0,2.0,2.0,16.333333,0.344898,19.733333,0.662162,15.900000,64.715520,108.023523,101.505949,0.717067,-0.083733,4.0,1.0,8.0,3.0,12.0,36.0,-5.0,45.0,0.0,0.004402,False,False,False,0.617,8,3,0,0,67.8,70.9,77,1.6,0.0
4,2003,1122,W13,27,0.740741,69.703704,67.703704,2.000000,12.584529,9.0,0.0,8.0,1.0,17.185185,0.349138,24.000000,0.691358,15.222222,63.424000,108.269069,104.582414,0.599878,0.140863,1.0,1.0,1.0,6.0,2.0,73.0,22.0,123.0,20.0,0.003029,False,False,False,0.522,2,0,0,0,73.5,62.7,77,1.2,0.0


## Tourney games

In [93]:
tourney_games_all = pd.read_csv('data/kaggle_data/MNCAATourneyCompactResults.csv')
tourney_games = tourney_games_all[tourney_games_all['Season'] >= 2003]
tourney_games.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1136,2003,134,1421,92,1411,84,N,1
1137,2003,136,1112,80,1436,51,N,0
1138,2003,136,1113,84,1272,71,N,0
1139,2003,136,1141,79,1166,73,N,0
1140,2003,136,1143,76,1301,74,N,1


In [94]:
tourney_games['rand'] = [random.random() for i in range(len(tourney_games))]
tourney_games['teams'] = [(t1, t2) if r >= 0.5 else (t2, t1) for t1, t2, r in \
                          zip(tourney_games['WTeamID'], tourney_games['LTeamID'], tourney_games['rand'])]
tourney_games['t1_score_diff'] = [t1-t2 if r >= 0.5 else t2-t1 for t1, t2, r in \
                          zip(tourney_games['WScore'], tourney_games['LScore'], tourney_games['rand'])]
tourney_games[['team1','team2']] = pd.DataFrame(tourney_games['teams'].tolist(), index=tourney_games.index)

In [95]:
tourney_games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,rand,teams,t1_score_diff,team1,team2
1136,2003,134,1421,92,1411,84,N,1,0.465952,"(1411, 1421)",-8,1411,1421
1137,2003,136,1112,80,1436,51,N,0,0.035274,"(1436, 1112)",-29,1436,1112
1138,2003,136,1113,84,1272,71,N,0,0.684712,"(1113, 1272)",13,1113,1272
1139,2003,136,1141,79,1166,73,N,0,0.431992,"(1166, 1141)",-6,1166,1141
1140,2003,136,1143,76,1301,74,N,1,0.945713,"(1143, 1301)",2,1143,1301
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,2021,148,1211,85,1425,66,N,0,0.882867,"(1211, 1425)",19,1211,1425
2313,2021,148,1417,51,1276,49,N,0,0.946065,"(1417, 1276)",2,1417,1276
2314,2021,152,1124,78,1222,59,N,0,0.848812,"(1124, 1222)",19,1124,1222
2315,2021,152,1211,93,1417,90,N,1,0.073905,"(1417, 1211)",-3,1417,1211


In [96]:
tourney_results = tourney_games[['Season', 'team1', 'team2', 't1_score_diff']]
tourney_results['is_t1_win'] = [diff > 0 for diff in tourney_results['t1_score_diff']]

In [97]:
tourney_results

,Season,team1,team2,t1_score_diff,is_t1_win
1136,2003,1411,1421,-8,False
1137,2003,1436,1112,-29,False
1138,2003,1113,1272,13,True
1139,2003,1166,1141,-6,False
1140,2003,1143,1301,2,True
...,...,...,...,...,...
2312,2021,1211,1425,19,True
2313,2021,1417,1276,2,True
2314,2021,1124,1222,19,True
2315,2021,1417,1211,-3,False


In [100]:
matchup_features_all = tourney_results.merge(features, left_on=['Season', 'team1'], right_on=['Season', 'TeamID'])\
.merge(features, left_on=['Season', 'team2'], right_on=['Season', 'TeamID'], suffixes=['_1', '_2'])
print(matchup_features_all.shape)
matchup_features_all.head()

(1217, 95)


,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,avg_height_1,avg_yr_1,preseason_pts_1,TeamID_2,Seed_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2,preseason_pts_2
0,2003,1411,1421,-8,False,1411,X16a,30,0.600000,72.800000,70.833333,1.966667,11.998565,7.0,3.0,5.0,2.0,18.500000,0.320721,28.066667,0.619952,15.233333,67.262791,108.724747,104.193929,0.594100,0.005900,0.0,0.0,1.0,11.0,1.0,217.0,-14.0,268.0,-15.0,0.002039,False,False,False,0.492,1,0,0,0,69.3,66.9,76,1.2,0.0,1421,X16b,29,0.448276,71.206897,78.448276,-7.241379,17.203959,3.0,2.0,6.0,3.0,18.000000,0.360153,20.931034,0.762768,16.206897,65.815172,106.461081,116.825884,0.206332,0.241944,0.0,0.0,6.0,10.0,6.0,186.0,-3.0,273.0,-7.0,0.001648,False,False,False,0.463,1,0,0,0,58.1,59.9,78,1.3,0.0
1,2003,1436,1112,-29,False,1436,Z16,29,0.655172,67.793103,63.137931,4.655172,12.382624,10.0,1.0,3.0,4.0,15.482759,0.340757,19.551724,0.657848,14.068966,62.374179,108.035787,100.640934,0.728997,-0.073825,0.0,0.0,1.0,9.0,1.0,144.0,41.0,144.0,38.0,0.003172,False,False,False,0.453,1,0,0,0,61.2,52.9,78,1.9,0.0,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.008013,True,False,False,0.738,24,12,5,1,98.4,99.0,77,1.6,1773.0
2,2003,1211,1112,-1,False,1211,Z09,31,0.741935,77.064516,68.193548,8.870968,12.945897,12.0,2.0,6.0,2.0,19.064516,0.375635,24.645161,0.721204,14.548387,65.507303,116.454416,102.380732,0.845678,-0.103742,5.0,3.0,5.0,3.0,10.0,43.0,3.0,41.0,-12.0,0.005134,False,False,True,0.784,4,2,0,0,NaN,NaN,77,NaN,381.0,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.008013,True,False,False,0.738,24,12,5,1,98.4,99.0,77,1.6,1773.0
3,2003,1242,1112,3,True,1242,Z02,30,0.766667,81.333333,66.366667,14.966667,17.827850,10.0,1.0,1.0,1.0,14.133333,0.339623,24.133333,0.665746,14.900000,70.337280,115.294347,94.655205,0.944210,-0.177543,9.0,3.0,6.0,1.0,15.0,6.0,5.0,4.0,2.0,0.007841,True,False,False,0.805,14,9,4,0,62.2,59.6,77,1.5,1715.0,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.008013,True,False,False,0.738,24,12,5,1,98.4,99.0,77,1.6,1773.0
4,2003,1113,1272,13,True,1113,Z10,29,0.620690,75.965517,69.172414,6.793103,16.889865,6.0,3.0,3.0,3.0,12.586207,0.317808,26.206897,0.669737,14.000000,65.671136,115.334912,103.952243,0.786348,-0.165659,4.0,2.0,10.0,1.0,14.0,32.0,-2.0,32.0,-8.0,0.004979,False,False,False,0.5

In [101]:
features.columns

Index(['Season', 'TeamID', 'Seed', 'total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'is_ap_pre_top_5', 'is_ap_pre_top_15', 'is_ap_pre_top_25',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr',
       'preseason_pts'],
      dtype='object')

In [102]:
difference_cols = ['total_games', 'win_pct', 'avg_points_for',
       'avg_points_against', 'avg_game_margin', 'std_game_margin',
       'non_home_wins', 'home_losses', 'close_wins', 'close_losses', '3FGA_pg',
       '3_pct', 'FTA_pg', 'FT_pct', 'TOV_pg', 'avg_pace', 'off_eff', 'def_eff',
       'pythag_win_pct', 'luck', 'wins_vs_tourney_teams',
       'away_wins_vs_tourney_teams', 'losses_vs_tourney_teams',
       'losses_vs_non_tourney_teams', 'games_vs_tourney_teams', 'final_net',
       'net_improvement', 'final_pom', 'pom_improvement', 'pi_i',
       'coach_WL_car', 'tourneys_car', 'sw16_car', 'ff_car', 'champ_car',
       'returning_min_pct', 'returning_score_pct', 'avg_height', 'avg_yr', 'preseason_pts']

In [103]:
for col in difference_cols:
    matchup_features_all['diff_'+col] = matchup_features_all[col+'_1'] - matchup_features_all[col+'_2']

In [104]:
pd.set_option("display.max_columns", None)
matchup_features_all.head()

,Season,team1,team2,t1_score_diff,is_t1_win,TeamID_1,Seed_1,total_games_1,win_pct_1,avg_points_for_1,avg_points_against_1,avg_game_margin_1,std_game_margin_1,non_home_wins_1,home_losses_1,close_wins_1,close_losses_1,3FGA_pg_1,3_pct_1,FTA_pg_1,FT_pct_1,TOV_pg_1,avg_pace_1,off_eff_1,def_eff_1,pythag_win_pct_1,luck_1,wins_vs_tourney_teams_1,away_wins_vs_tourney_teams_1,losses_vs_tourney_teams_1,losses_vs_non_tourney_teams_1,games_vs_tourney_teams_1,final_net_1,net_improvement_1,final_pom_1,pom_improvement_1,pi_i_1,is_ap_pre_top_5_1,is_ap_pre_top_15_1,is_ap_pre_top_25_1,coach_WL_car_1,tourneys_car_1,sw16_car_1,ff_car_1,champ_car_1,returning_min_pct_1,returning_score_pct_1,avg_height_1,avg_yr_1,preseason_pts_1,TeamID_2,Seed_2,total_games_2,win_pct_2,avg_points_for_2,avg_points_against_2,avg_game_margin_2,std_game_margin_2,non_home_wins_2,home_losses_2,close_wins_2,close_losses_2,3FGA_pg_2,3_pct_2,FTA_pg_2,FT_pct_2,TOV_pg_2,avg_pace_2,off_eff_2,def_eff_2,pythag_win_pct_2,luck_2,wins_vs_tourney_teams_2,away_wins_vs_tourney_teams_2,losses_vs_tourney_teams_2,losses_vs_non_tourney_teams_2,games_vs_tourney_teams_2,final_net_2,net_improvement_2,final_pom_2,pom_improvement_2,pi_i_2,is_ap_pre_top_5_2,is_ap_pre_top_15_2,is_ap_pre_top_25_2,coach_WL_car_2,tourneys_car_2,sw16_car_2,ff_car_2,champ_car_2,returning_min_pct_2,returning_score_pct_2,avg_height_2,avg_yr_2,preseason_pts_2,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr,diff_preseason_pts
0,2003,1411,1421,-8,False,1411,X16a,30,0.600000,72.800000,70.833333,1.966667,11.998565,7.0,3.0,5.0,2.0,18.500000,0.320721,28.066667,0.619952,15.233333,67.262791,108.724747,104.193929,0.594100,0.005900,0.0,0.0,1.0,11.0,1.0,217.0,-14.0,268.0,-15.0,0.002039,False,False,False,0.492,1,0,0,0,69.3,66.9,76,1.2,0.0,1421,X16b,29,0.448276,71.206897,78.448276,-7.241379,17.203959,3.0,2.0,6.0,3.0,18.000000,0.360153,20.931034,0.762768,16.206897,65.815172,106.461081,116.825884,0.206332,0.241944,0.0,0.0,6.0,10.0,6.0,186.0,-3.0,273.0,-7.0,0.001648,False,False,False,0.463,1,0,0,0,58.1,59.9,78,1.3,0.0,1,0.151724,1.593103,-7.614943,9.208046,-5.205395,4.0,1.0,-1.0,-1.0,0.500000,-0.039433,7.135632,-0.142815,-0.973563,1.447619,2.263666,-12.631956,0.387769,-0.236045,0.0,0.0,-5.0,1.0,-5.0,31.0,-11.0,-5.0,-8.0,0.000391,0.029,0,0,0,0,11.2,7.0,-2,-0.1,0.0
1,2003,1436,1112,-29,False,1436,Z16,29,0.655172,67.793103,63.137931,4.655172,12.382624,10.0,1.0,3.0,4.0,15.482759,0.340757,19.551724,0.657848,14.068966,62.374179,108.035787,100.640934,0.728997,-0.073825,0.0,0.0,1.0,9.0,1.0,144.0,41.0,144.0,38.0,0.003172,False,False,False,0.453,1,0,0,0,61.2,52.9,78,1.9,0.0,1112,Z01,28,0.892857,85.214286,70.250000,14.964286,12.330225,11.0,1.0,3.0,2.0,20.071429,0.350534,25.000000,0.701429,14.785714,72.569829,116.575300,96.408747,0.936204,-0.043347,10.0,5.0,2.0,1.0,12.0,2.0,6.0,3.0,0.0,0.008013,True,False,False,0.738,24,12,5,1,98.4,99.0,77,1.6,1773.0,1,-0.237685,-17.421182,-7.112069,-10.309113,0.052399,-1.0,0.0,0.0,2.0,-4.588670,-0.009777,-5.448276,-0.043580,-0.716749,-10.195649,-8.539513,4.232186,-0.207207,-0.030478,-10.0,-5.0,-1.0,8.0,-11.0,142.0,35.0,141.0,38.0,-0.004841,-0.285,-23,-12,-5,-1,-37.2,-46.1,1,0.3,-1773.0
2,2003,1211,1112,-1,False,1211,Z09,31,0.741935,77.064516,68.193548,8.870968,12.945897,12.0,2.0,6.0,2.0,19.064516,0.375635,24.645161,0.721204,14.548387,65.507303,116.454416,102.380732,0.845678,-0

In [105]:
diff_cols = [col for col in matchup_features_all.columns if 'diff' in col]
matchup_cols = ['Season', 'TeamID_1', 'TeamID_2', 'Seed_1', 'Seed_2'] + diff_cols

In [106]:
matchup_features_basic = matchup_features_all[matchup_cols]

In [107]:
print(matchup_features_basic.shape)

(1217, 46)


In [108]:
matchup_features_basic.to_csv('data/model_data/matchup_features.csv', index=False)

In [64]:
matchup_features_basic.head()

,TeamID_1,TeamID_2,Seed_1,Seed_2,t1_score_diff,diff_total_games,diff_win_pct,diff_avg_points_for,diff_avg_points_against,diff_avg_game_margin,diff_std_game_margin,diff_non_home_wins,diff_home_losses,diff_close_wins,diff_close_losses,diff_3FGA_pg,diff_3_pct,diff_FTA_pg,diff_FT_pct,diff_TOV_pg,diff_avg_pace,diff_off_eff,diff_def_eff,diff_pythag_win_pct,diff_luck,diff_wins_vs_tourney_teams,diff_away_wins_vs_tourney_teams,diff_losses_vs_tourney_teams,diff_losses_vs_non_tourney_teams,diff_games_vs_tourney_teams,diff_final_net,diff_net_improvement,diff_final_pom,diff_pom_improvement,diff_pi_i,diff_coach_WL_car,diff_tourneys_car,diff_sw16_car,diff_ff_car,diff_champ_car,diff_returning_min_pct,diff_returning_score_pct,diff_avg_height,diff_avg_yr,diff_preseason_pts
0,1272,1113,Z07,Z10,-13,0,0.172414,-1.448276,-3.344828,1.896552,-6.116003,3.0,-1.0,-1.0,1.0,7.482759,0.030989,-3.310345,-0.016122,-0.206897,1.004653,-4.182093,-5.603289,0.062388,0.110026,0.0,0.0,-7.0,2.0,-7.0,-6.0,16.0,-12.0,37.0,0.000396,0.201,3,3,1,0,NaN,NaN,-1,NaN,-2
1,1242,1113,Z02,Z10,32,1,0.145977,5.367816,-2.805747,8.173563,0.937985,4.0,-2.0,-2.0,-2.0,1.547126,0.021814,-2.073563,-0.003991,0.900000,4.666144,-0.040565,-9.297038,0.157862,-0.011885,5.0,1.0,-4.0,0.0,1.0,-26.0,7.0,-28.0,10.0,0.002862,0.285,11,9,4,0,-11.0,-7.0,0,-0.1,1711
2,1143,1301,W08,W09,2,-1,0.124138,2.082759,1.758621,0.324138,-5.455370,3.0,-2.0,0.0,0.0,-5.465517,0.022444,-0.949425,-0.084846,-0.027586,2.833959,-2.167815,-2.687209,0.008112,0.116026,2.0,0.0,-2.0,-2.0,0.0,-20.0,-20.0,-8.0,-26.0,0.000024,-0.001,4,2,0,0,-10.0,-0.1,-2,0.1,-38
3,1143,1328,W08,W01,-9,-1,-0.075862,3.316092,9.591954,-6.275862,-3.906324,-1.0,0.0,-3.0,1.0,-1.932184,-0.017155,0.917241,-0.022373,2.372414,5.257924,-3.829212,7.029510,-0.198479,0.122617,-4.0,-2.0,-1.0,3.0,-5.0,30.0,-10.0,31.0,-15.0,-0.002071,-0.029,-3,-1,-1,0,-15.3,-5.2,-1,0.3,-1637
4,1393,1328,W03,W01,16,-1,0.027586,8.936782,9.729885,-0.793103,-0.450743,-2.0,-1.0,-1.0,0.0,-3.104598,-0.063238,5.020690,-0.014455,1.820690,7.240673,1.150369,5.033997,-0.042346,0.069932,-3.0,-1.0,-2.0,1.0,-5.0,6.0,3.0,6.0,2.0,-0.000715,0.110,12,9,2,1,-16.3,-17.1,2,-0.4,-1561
